<a href="https://colab.research.google.com/github/FBehrad/Charm/blob/main/charm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Get requirements.py**


In [1]:
!wget https://raw.githubusercontent.com/FBehrad/Charm/refs/heads/main/requirements.txt

--2025-04-11 08:48:08--  https://raw.githubusercontent.com/FBehrad/Charm/refs/heads/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 312 [text/plain]
Saving to: ‘requirements.txt.1’

requirements.txt.1  100%[===================>]     312  --.-KB/s    in 0s      

2025-04-11 08:48:08 (17.8 MB/s) - ‘requirements.txt.1’ saved [312/312]





---


**2. Install the rquirements:**


In [2]:
!pip install -r requirements.txt
!pip install "numpy<2"

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117




---
**3. Install Charm tokenizer**


In [3]:
!pip install Charm-tokenizer==1.0.16



---

**4. Prepare patches and their corresponding position embeddings**

Charm Tokenizer has the following input args:

*   patch_selection (str): The method for selecting important patches: Options: 'saliency', 'random', 'frequency', 'gradient', 'entropy', 'original'.
*   training_dataset (str): Used to set the number of ViT input tokens to match a specific training dataset from the paper.
  * Aesthetic assessment datasets: 'aadb', 'tad66k', 'para', 'baid'.
  * Quality assessment datasets: 'spaq', 'koniq10k'.

* backbone (str): The ViT backbone model (default: 'facebook/dinov2-small').
* factor (float): The downscaling factor for less important patches (default: 0.5).
* scales (int): The number of scales used for multiscale processing (default: 2).
* random_crop_size (tuple): Used for the 'original' patch selection strategy (default: (224, 224)).
* downscale_shortest_edge (int): Used for the 'original' patch selection strategy (default: 256).
* without_pad_or_dropping (bool): Whether to avoid padding or dropping patches (default: True).


The output is the preprocessed tokens, their corresponding positional embeddings, and a mask token that indicates which patches are in high resolution and which are in low resolution.


**Note:** While random patch selection during training helps avoid overfitting,for consistent results during inference, fully deterministic patch selection approaches should be used.

In [4]:
from Charm_tokenizer.ImageProcessor import Charm_Tokenizer
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
img_path = requests.get(url, stream=True).raw

charm_tokenizer = Charm_Tokenizer(patch_selection='frequency', training_dataset='tad66k', without_pad_or_dropping=True)
tokens, pos_embed, mask_token = charm_tokenizer.preprocess(img_path)



---
**5. Predict aesthetic score**

* If training_dataset is set to 'spaq' or 'koniq10k', the model predicts the image *quality* score. For other options ('aadb', 'tad66k', 'para', 'baid'), it predicts the image *aesthetic* score.

* Selecting a dataset with image resolutions similar to your input images can improve prediction accuracy.

* For more details about the process, please refer to the paper.



In [5]:
from Charm_tokenizer.Backbone import backbone
from Charm_tokenizer import model

model = backbone(training_dataset='tad66k', device='cpu')
prediction = model.predict(tokens, pos_embed, mask_token)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tad66k_charm.pth:   0%|          | 0.00/86.3M [00:00<?, ?B/s]

Prediction: 3.813 from 1 to 10
